In [ ]:
import pandas as pd

# Define hierarchies
degree_hierarchy = {
    'none': 0,
    'apprenticeship': 1,
    'bachelor': 2,
    'master': 3,
    'doctorate': 4
}

seniority_hierarchy = {
    'none': 0,
    'junior': 1,
    'midlevel': 2,
    'senior': 3
}

# Example data
talents = [
    {
        "languages": [{"title": "English", "rating": "C2"}, {"title": "German", "rating": "B2"}],
        "job_roles": ["software engineer", "data scientist"],
        "seniority": "midlevel",
        "salary_expectation": 80000,
        "degree": "master"
    },
    {
        "languages": [{"title": "English", "rating": "B2"}, {"title": "Spanish", "rating": "C1"}],
        "job_roles": ["data analyst"],
        "seniority": "junior",
        "salary_expectation": 50000,
        "degree": "bachelor"
    }
]

jobs = [
    {
        "languages": [{"title": "English", "rating": "C1", "must_have": True}],
        "job_roles": ["software engineer"],
        "seniorities": ["midlevel", "senior"],
        "max_salary": 90000,
        "min_degree": "bachelor"
    },
    {
        "languages": [{"title": "Spanish", "rating": "B2", "must_have": True}],
        "job_roles": ["data analyst"],
        "seniorities": ["junior", "midlevel"],
        "max_salary": 60000,
        "min_degree": "bachelor"
    }
]

# Convert to DataFrame
talent_df = pd.DataFrame(talents)
job_df = pd.DataFrame(jobs)

# Define hierarchies
degree_hierarchy = {
    'none': 0,
    'apprenticeship': 1,
    'bachelor': 2,
    'master': 3,
    'doctorate': 4
}

seniority_hierarchy = {
    'none': 0,
    'junior': 1,
    'midlevel': 2,
    'senior': 3
}

# Encode the degrees
talent_df['degree_encoded'] = talent_df['degree'].map(degree_hierarchy)
job_df['min_degree_encoded'] = job_df['min_degree'].map(degree_hierarchy)

# Encode the seniorities
talent_df['seniority_encoded'] = talent_df['seniority'].map(seniority_hierarchy)
job_df['min_seniority_encoded'] = job_df['seniorities'].apply(lambda x: min([seniority_hierarchy[sen] for sen in x]))

# Function to filter and combine
def filter_and_combine(talents, jobs):
    def meets_language_requirements(talent_languages, job_languages):
        for job_lang in job_languages:
            if job_lang.get('must_have', False):
                match = next((talent_lang for talent_lang in talent_languages if talent_lang['title'] == job_lang['title'] and talent_lang['rating'] >= job_lang['rating']), None)
                if not match:
                    return False
        return True

    def meets_seniority_requirements(talent_seniority_encoded, job_min_seniority_encoded):
        return talent_seniority_encoded >= job_min_seniority_encoded

    filtered_talents = []
    for talent in talents:
        for job in jobs:
            if meets_language_requirements(talent['languages'], job['languages']) and meets_seniority_requirements(talent['seniority_encoded'], job['min_seniority_encoded']):
                combined_data = {**talent, **job}
                filtered_talents.append(combined_data)

    df = pd.DataFrame(filtered_talents)
    return df

# Apply the function
df = filter_and_combine(talent_df.to_dict('records'), job_df.to_dict('records'))
print(df)

In [53]:
import json
from typing import List, Dict
import pandas as pd

def load_data(file_path: str) -> List[Dict]:
    with open(file_path, 'r') as f:
        return json.load(f)

def clean_data(data: List[Dict]) -> List[Dict]:
    # Implement data cleaning logic
    # For example, handling missing values, standardizing formats, etc.
    return data

data = load_data('data.json')

In [54]:
# Create records
records = []

for item in data:
    talent = item["talent"]
    job = item["job"]
    label = item["label"]

    record = {
        "talent_degree": talent["degree"],
        "talent_job_roles": talent["job_roles"],
        "talent_languages": talent["languages"],
        "talent_salary_expectation": talent["salary_expectation"],
        "talent_seniority": talent["seniority"],
        "job_roles": job["job_roles"],
        "job_languages": job["languages"],
        "job_max_salary": job["max_salary"],
        "job_min_degree": job["min_degree"],
        "job_seniorities": job["seniorities"],
        "label": label
    }

    records.append(record)

# Talent DataFrame
talent_df = pd.DataFrame([
    {
        "degree": item["talent"]["degree"],
        "job_roles": item["talent"]["job_roles"],
        "languages": item["talent"]["languages"],
        "salary_expectation": item["talent"]["salary_expectation"],
        "seniority": item["talent"]["seniority"]
    }
    for item in data
])

# Job DataFrame
job_df = pd.DataFrame([
    {
        "job_roles": item["job"]["job_roles"],
        "languages": item["job"]["languages"],
        "max_salary": item["job"]["max_salary"],
        "min_degree": item["job"]["min_degree"],
        "seniorities": item["job"]["seniorities"]
    }
    for item in data
])

# Labels DataFrame
labels_df = pd.DataFrame([
    {"label": item["label"]}
    for item in data
])


###### EXPLORATORY ANLYSIS ######
# Function to extract unique languages and levels
def get_unique_languages_and_levels(talent_df, job_df):
    # Extract languages and levels from talent_df
    talent_languages = [lang for sublist in talent_df['languages'] for lang in sublist]
    
    # Extract languages and levels from job_df
    job_languages = [lang for sublist in job_df['languages'] for lang in sublist]
    
    # Combine both lists
    combined_languages = talent_languages + job_languages
    
    # Get unique languages and levels
    unique_languages_and_levels = { (lang['title'], lang['rating']) for lang in combined_languages }
    
    # Separate languages and levels into two lists
    languages = [lang for lang, _ in unique_languages_and_levels]
    levels = [level for _, level in unique_languages_and_levels]
    
    return languages, levels

# Get unique languages and levels
languages, levels = get_unique_languages_and_levels(talent_df, job_df)
print("Languages:", set(languages))
print("Levels:", set(levels))

def get_unique_languages_and_levels(talent_df, job_df):
    # Extract languages and levels from talent_df
    talent_languages = [lang for sublist in talent_df['languages'] for lang in sublist]
    
    # Extract languages and levels from job_df
    job_languages = [lang for sublist in job_df['languages'] for lang in sublist]
    
    # Combine both lists
    combined_languages = talent_languages + job_languages
    
    # Get unique languages and levels
    unique_languages_and_levels = { (lang['title'], lang['rating']) for lang in combined_languages }
    
    # Create a dictionary with languages as keys and levels as values
    languages_dict = {}
    for lang, level in unique_languages_and_levels:
        if lang not in languages_dict:
            languages_dict[lang] = []
        languages_dict[lang].append(level)
    
    return languages_dict

# Get unique languages and levels
languages_dict = get_unique_languages_and_levels(talent_df, job_df)
print(languages_dict)


# Print unique values of 'min_degree' from job_df
print("\n\n")
unique_min_degrees = job_df['min_degree'].unique()
print("Unique 'min_degree' values from job_df:")
print(unique_min_degrees)

# Print unique values of 'degree' from talent_df
unique_degrees = talent_df['degree'].unique()
print("Unique 'degree' values from talent_df:")
print(unique_degrees)

print("\n\n")
unique_seniorities = talent_df['seniority'].unique()
print("Unique 'degree' values from talent_df:")
print(unique_seniorities)
# unique_seniorities = job_df['seniorities'].unique()
# print("Unique 'degree' values from talent_df:")
# print(unique_seniorities)


###### EXPLORATORY ANLYSIS ######

Languages: {'Tamil', 'Japanese', 'Greek', 'Slovak', 'Arabic', 'Russian', 'Swedish', 'Portuguese', 'Hungarian', 'Bengalese', 'English', 'Danish', 'Czech', 'Turkish', 'Norwegian', 'Finnish', 'Hindi', 'Persian', 'Hebrew', 'Korean', 'Bulgarian', 'Latvian', 'Macedonian', 'Albanian', 'Croatian', 'Polish', 'Romanian', 'Serbian', 'German', 'Armenian', 'Italian', 'Spanish', 'French', 'Dutch', 'Chinese'}
Levels: {'C2', 'A1', 'A2', 'B1', 'C1', 'B2'}
{'Arabic': ['C2', 'C1', 'A1', 'B1', 'A2', 'B2'], 'Polish': ['B1', 'A1', 'A2', 'B2', 'C2', 'C1'], 'Hebrew': ['B1', 'A2', 'B2', 'C1'], 'Korean': ['C1', 'A1', 'A2'], 'Persian': ['A2', 'C2', 'C1', 'A1', 'B1'], 'French': ['C1', 'A1', 'B1', 'A2', 'B2', 'C2'], 'Hungarian': ['C2', 'C1', 'A2'], 'Japanese': ['A1', 'B1', 'A2', 'B2', 'C2', 'C1'], 'Russian': ['A1', 'A2', 'B2', 'C2', 'C1', 'B1'], 'Dutch': ['A1', 'B1', 'A2', 'B2', 'C2', 'C1'], 'Portuguese': ['C1', 'A1', 'B1', 'A2', 'B2', 'C2'], 'Czech': ['A1', 'B1', 'C2'], 'Serbian': ['C2', 'C1', 'B1', 'B2'], 'Spani

In [55]:
df.languages.iloc[0].iloc[1]

[{'title': 'German', 'rating': 'C1', 'must_have': True},
 {'title': 'English', 'rating': 'B2', 'must_have': True}]

In [56]:
df.languages.iloc[0].iloc[1]

[{'title': 'German', 'rating': 'C1', 'must_have': True},
 {'title': 'English', 'rating': 'B2', 'must_have': True}]

In [61]:
#### CHEKC IF FILTERED DATAFRAME HAS ALL MUST_HAVE LANGUAGES ####
def check_language_requirements(df):
    """ This is to valudate hypotesis that all canidats have all must_have languages 
    and that it can be a filter to select the best candidates"""
    def rating_to_level(rating):
        levels = {'A1': 1, 'A2': 2, 'B1': 3, 'B2': 4, 'C1': 5, 'C2': 6}
        return levels.get(rating, 0)
    
    def has_required_languages(candidate_languages, job_languages):
        candidate_dict = {lang['title']: rating_to_level(lang['rating']) for lang in candidate_languages}
        for job_lang in job_languages:
            if job_lang.get('must_have', False):
                job_title = job_lang['title']
                job_rating = rating_to_level(job_lang['rating'])
                if candidate_dict.get(job_title, 0) < job_rating:
                    return False
        return True
    
    for index, row in df[df['label'] == True].iterrows():
        candidate_languages = row['languages'].iloc[0]
        job_languages = row['languages'].iloc[1]
        if not has_required_languages(candidate_languages, job_languages):
            return False
    return True

# Example usage
result = check_language_requirements(df)
print("All rows with label=True haintve all must_have criteria fulfilled:", result)


# Define hierarchies
degree_hierarchy = {
    'none': 0,
    'apprenticeship': 1,
    'bachelor': 2,
    'master': 3,
    'doctorate': 4
}

seniority_hierarchy = {
    'none': 0,
    'junior': 1,
    'midlevel': 2,
    'senior': 3
}


# Encode the degrees
talent_df['degree_encoded'] = talent_df['degree'].map(degree_hierarchy)
job_df['min_degree_encoded'] = job_df['min_degree'].map(degree_hierarchy)

# Encode the seniorities
talent_df['seniority_encoded'] = talent_df['seniority'].map(seniority_hierarchy)
job_df['min_seniority_encoded'] = job_df['seniorities'].apply(lambda x: min([seniority_hierarchy[sen] for sen in x]))

# join the dataframes on index 
df = pd.concat([talent_df, job_df, labels_df], axis=1)
df.head(3)

# Function to check requirements

non_compliant_rows = []

for index, row in df[df['label'] == True].iterrows():
    candidate_max_salary = row['salary_expectation']
    job_max_salary = row['max_salary']
    if candidate_max_salary > job_max_salary:
        reason = 'Salary expectation too high'
        non_compliant_rows.append({**row, 'reason': reason})
    elif not any(role in row['job_roles'].iloc[0] for role in row['job_roles'].iloc[1]):
        reason = 'No matching job roles'
        non_compliant_rows.append({**row, 'reason': reason})
    elif row['degree_encoded'] < row['min_degree_encoded']:
        reason = 'Degree not sufficient'
        non_compliant_rows.append({**row, 'reason': reason})
    elif row['seniority_encoded'] < row['min_seniority_encoded']:
        reason = 'Seniority not sufficient'
        non_compliant_rows.append({**row, 'reason': reason})

non_compliant_df = pd.DataFrame(non_compliant_rows)
non_compliant_df

# Only salary too high for job found that it cannot be a filter since it is not a must have
# but we can filter by max salary, must_have_laganuge, degree and seniority

All rows with label=True haintve all must_have criteria fulfilled: True


,degree,job_roles,languages,salary_expectation,seniority,degree_encoded,seniority_encoded,max_salary,min_degree,seniorities,min_degree_encoded,min_seniority_encoded,label,reason
0,bachelor,"job_roles [c-c-developer, qa-engineer, devo...","languages [{'rating': 'C2', 'title': 'Germa...",70000,midlevel,2,2,65000,none,"[junior, midlevel, senior]",0,1,True,Salary expectation too high
1,bachelor,"job_roles [full-stack-developer, java-devel...","languages [{'rating': 'C2', 'title': 'Germa...",71000,midlevel,2,2,70000,bachelor,"[junior, midlevel, senior]",2,1,True,Salary expectation too high
2,master,"job_roles [sales-manager, sales-engineer, k...","languages [{'rating': 'C2', 'title': 'Germa...",69000,junior,3,1,65000,bachelor,[junior],2,1,True,Salary expectation too high
3,bachelor,"job_roles [backend-developer, full-stack-de...","languages [{'rating': 'B2', 'title': 'Chine...",110000,midlevel,2,2,100000,none,"[midlevel, senior]",0,2,True,Salary expectation too high
4,master,"job_roles [full-stack-developer, java-devel...","languages [{'rating': 'C2', 'title': 'Germa...",95000,senior,3,3,90000,apprenticeship,"[midlevel, senior]",1,2,True,Salary expectation too high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,master,"job_roles [frontend-developer, full-stack-d...","languages [{'rating': 'C2', 'title': 'Germa...",90000,midlevel,3,2,82000,bachelor,"[none, junior, midlevel, senior]",2,0,True,Salary expectation too high
183,master,"job_roles [full-stack-developer, software-a...","languages [{'rating': 'C2', 'title': 'Arabi...",100000,senior,3,3,90000,apprenticeship,"[midlevel, senior]",1,2,True,Salary expectation too high
184,master,"job_roles [backend-developer, software-arch...","languages [{'rating': 'C2', 'title': 'Germa...",105000,senior,3,3,100000,none,"[midlevel, senior]",0,2,True,Salary expectation too high
185,none,"job_roles [data-scientist, data-analyst, bu...","languages [{'rating': 'C2', 'title': 'Germa...",66000,midlevel,0,2,65000,none,"[midlevel, senior]",0,2,True,Salary expectation too high


In [62]:
non_compliant_df.reason.unique()

array(['Salary expectation too high'], dtype=object)

In [44]:
df.head(1)

,degree,job_roles,languages,salary_expectation,seniority,job_roles,languages,max_salary,min_degree,seniorities,label
0,bachelor,"[frontend-developer, backend-developer, full-s...","[{'rating': 'C2', 'title': 'German'}, {'rating...",48000,junior,[frontend-developer],"[{'title': 'German', 'rating': 'C1', 'must_hav...",70000,none,"[junior, midlevel]",True


In [ ]:
# Define hierarchies
degree_hierarchy = {
    'none': 0,
    'apprenticeship': 1,
    'bachelor': 2,
    'master': 3,
    'doctorate': 4
}

seniority_hierarchy = {
    'none': 0,
    'junior': 1,
    'midlevel': 2,
    'senior': 3
}

# Encode the degrees
talent_df['degree_encoded'] = talent_df['degree'].map(degree_hierarchy)
job_df['min_degree_encoded'] = job_df['min_degree'].map(degree_hierarchy)

# Encode the seniorities
talent_df['seniority_encoded'] = talent_df['seniority'].map(seniority_hierarchy)
job_df['min_seniority_encoded'] = job_df['seniorities'].apply(lambda x: min([seniority_hierarchy[sen] for sen in x]))


In [43]:
# join the dataframes on index 
df = pd.concat([talent_df, job_df, labels_df], axis=1)
df.head(3)

,degree,job_roles,languages,salary_expectation,seniority,job_roles,languages,max_salary,min_degree,seniorities,label
0,bachelor,"[frontend-developer, backend-developer, full-s...","[{'rating': 'C2', 'title': 'German'}, {'rating...",48000,junior,[frontend-developer],"[{'title': 'German', 'rating': 'C1', 'must_hav...",70000,none,"[junior, midlevel]",True
1,master,"[frontend-developer, full-stack-developer, c-c...","[{'rating': 'C2', 'title': 'German'}, {'rating...",44000,junior,[frontend-developer],"[{'title': 'German', 'rating': 'C1', 'must_hav...",70000,none,"[junior, midlevel]",True
2,none,"[frontend-developer, backend-developer, php-de...","[{'rating': 'C2', 'title': 'English'}, {'ratin...",40000,senior,[php-developer],"[{'title': 'German', 'rating': 'C1', 'must_hav...",65000,none,"[midlevel, senior]",True


In [16]:
talent_df.head(2)

,degree,job_roles,languages,salary_expectation,seniority
0,bachelor,"[frontend-developer, backend-developer, full-s...","[{'rating': 'C2', 'title': 'German'}, {'rating...",48000,junior
1,master,"[frontend-developer, full-stack-developer, c-c...","[{'rating': 'C2', 'title': 'German'}, {'rating...",44000,junior


In [17]:

# Standardizing text data
def standardize_text_columns(df, columns):
    for col in columns:
        df[col] = df[col].str.lower().str.strip()

standardize_text_columns(talent_df, ['seniority', 'degree'])
standardize_text_columns(job_df, ['min_degree'])


############################# Encoding Categorical Data #############################
# Define the degree levels and encoding
degree_hierarchy = {
    'none': 0,
    'apprenticeship': 1,
    'bachelor': 2,
    'master': 3,
    'doctorate': 4
}

# Encode the degrees
talent_df['degree_encoded'] = talent_df['degree'].map(degree_hierarchy)
job_df['min_degree_encoded'] = job_df['min_degree'].map(degree_hierarchy)


# # Create binary columns for cumulative qualifications
# talents_df['has_bachelor_or_higher'] = talents_df['degree_encoded'] >= degree_hierarchy['bachelor']
# talents_df['has_master_or_higher'] = talents_df['degree_encoded'] >= degree_hierarchy['master']
# talents_df['has_doctorate'] = talents_df['degree_encoded'] == degree_hierarchy['doctorate']

# # Optional: Create similar binary columns for job requirements if needed
# jobs_df['requires_bachelor_or_higher'] = jobs_df['min_degree_encoded'] >= degree_hierarchy['bachelor']
# jobs_df['requires_master_or_higher'] = jobs_df['min_degree_encoded'] >= degree_hierarchy['master']
# jobs_df['requires_doctorate'] = jobs_df['min_degree_encoded'] == degree_hierarchy['doctorate']


Unique 'min_degree' values from job_df:
['none' 'bachelor' 'master' 'apprenticeship' 'doctorate']
Unique 'degree' values from talent_df:
['bachelor' 'master' 'none' 'apprenticeship' 'doctorate']


In [ ]:
job_df['seniorities'].apply(lambda x: 'none' or None in x)

In [24]:
none_examples

,job_roles,languages,max_salary,min_degree,seniorities,min_degree_encoded,has_none


In [23]:
# # Convert each row in the 'seniorities' column to a list
# job_df['seniorities'] = job_df['seniorities'].apply(lambda x: x.split(','))

# # Convert each row in the 'job_roles' column to a list
# job_df['job_roles'] = job_df['job_roles'].apply(lambda x: x.split(','))

# Create a new column 'has_none' to indicate whether 'none' was present in 'seniorities'
job_df['has_none'] = job_df['seniorities'].apply(lambda x: 'none' in x or None in x)

# Remove 'none' from the 'seniorities' lists
job_df['seniorities'] = job_df['seniorities'].apply(lambda x: [level for level in x if level != 'none' and level is not None])

# Explore examples where there is 'none'
none_examples = job_df[job_df['seniorities'].apply(lambda x: 'none' in x)]

print("Transformed 'seniorities' column:")
print(job_df['seniorities'].head())

print("\nNew 'has_none' column:")
print(job_df['has_none'].head())

print("\nExamples where 'none' was present:")
none_examples = job_df[job_df['has_none']]
job_df.sample(10)

Transformed 'seniorities' column:
0    [junior, midlevel]
1    [junior, midlevel]
2    [midlevel, senior]
3    [junior, midlevel]
4            [midlevel]
Name: seniorities, dtype: object

New 'has_none' column:
0    False
1    False
2    False
3    False
4    False
Name: has_none, dtype: bool

Examples where 'none' was present:


,job_roles,languages,max_salary,min_degree,seniorities,min_degree_encoded,has_none
1522,[data-engineer],"[{'title': 'German', 'rating': 'C1', 'must_hav...",80000,master,"[midlevel, senior]",3,False
714,[frontend-developer],"[{'title': 'German', 'rating': 'C1', 'must_hav...",70000,none,[midlevel],0,False
1458,[sales-manager],"[{'title': 'German', 'rating': 'C2', 'must_hav...",80000,bachelor,[senior],2,False
1479,[sales-manager],"[{'title': 'German', 'rating': 'C1', 'must_hav...",70000,bachelor,"[junior, midlevel]",2,False
1633,[frontend-developer],"[{'title': 'German', 'rating': 'C1', 'must_hav...",70000,none,"[junior, midlevel]",0,False
1116,[frontend-developer],"[{'title': 'German', 'rating': 'C1', 'must_hav...",70000,master,"[junior, midlevel]",3,False
241,[frontend-developer],"[{'title': 'German', 'rating': 'C1', 'must_hav...",70000,none,"[junior, midlevel]",0,False
696,[full-stack-developer],"[{'title': 'German', 'rating': 'B2', 'must_hav...",90000,apprenticeship,"[midlevel, senior]",1,False
66,[consulting],"[{'title': 'German', 'rating': 'C2', 'must_hav...",75000,none,"[midlevel, senior]",0,False
1104,[mobile-developer],"[{'title': 'German', 'rating': 'C1', 'must_hav...",77000,doctorate,"[midlevel, senior]",4,False


In [51]:
# Explore examples where there is 'none' in 'job_roles'
none_examples_job_roles = job_df[job_df['job_roles'].apply(lambda x: 'none' in x)]
none_examples_job_roles

,job_roles,languages,max_salary,min_degree,seniorities


In [26]:
talent_df.languages.iloc[0]

[{'rating': 'C2', 'title': 'German'},
 {'rating': 'C2', 'title': 'English'},
 {'rating': 'B2', 'title': 'French'},
 {'rating': 'A2', 'title': 'Turkish'}]

In [1]:
job_df.laguages.iloc[0]

NameError: name 'job_df' is not defined

In [ ]:
labels_df

In [ ]:
print("Talents DataFrame:")
print(talents_df.info())
print("\nJobs DataFrame:")
print(jobs_df.info())
print("\nLabels DataFrame:")
print(labels_df.info())


print("\nTalents Summary Statistics:")
print(talents_df.describe())
print("\nJobs Summary Statistics:")
print(jobs_df.describe())
print("\nLabels Summary Statistics:")
print(labels_df.describe())

In [10]:
import pandas as pd
import json

# Assuming the JSON data is stored in a variable called 'json_data'
data = json_data

# Function to flatten language data
def flatten_languages(languages, prefix=''):
    flattened = {}
    for i, lang in enumerate(languages, 1):
        flattened[f'{prefix}language_{i}_title'] = lang['title']
        flattened[f'{prefix}language_{i}_rating'] = lang['rating']
        if 'must_have' in lang:
            flattened[f'{prefix}language_{i}_must_have'] = lang['must_have']
    return flattened

# Prepare talents data
talents = []
for item in data:
    talent = item['talent']
    talent['label'] = item['label']
    talent.update(flatten_languages(talent['languages']))
    talent['job_roles'] = ','.join(talent['job_roles'])
    talents.append(talent)

# Create talents DataFrame
talents_df = pd.DataFrame(talents)

# Prepare jobs data
jobs = []
for item in data:
    job = item['job']
    job['label'] = item['label']
    job.update(flatten_languages(job['languages'], 'required_'))
    job['job_roles'] = ','.join(job['job_roles'])
    job['seniorities'] = ','.join(job['seniorities'])
    jobs.append(job)

# Create jobs DataFrame
jobs_df = pd.DataFrame(jobs)

# Reorder columns for better readability
talent_columns = ['label', 'seniority', 'degree', 'salary_expectation', 'job_roles'] + \
                 [col for col in talents_df.columns if col.startswith('language_')]
talents_df = talents_df[talent_columns]

job_columns = ['label', 'job_roles', 'seniorities', 'max_salary', 'min_degree'] + \
              [col for col in jobs_df.columns if col.startswith('required_language_')]
jobs_df = jobs_df[job_columns]

In [14]:
jobs_df.describe()

,max_salary
count,2000.000000
mean,72658.000000
std,9237.678461
min,45000.000000
25%,70000.000000
50%,70000.000000
75%,80000.000000
max,100000.000000


In [11]:
talents_df

,label,seniority,degree,salary_expectation,job_roles,language_1_title,language_1_rating,language_2_title,language_2_rating,language_3_title,...,language_4_title,language_4_rating,language_5_title,language_5_rating,language_6_title,language_6_rating,language_7_title,language_7_rating,language_8_title,language_8_rating
0,True,junior,bachelor,48000,"frontend-developer,backend-developer,full-stac...",German,C2,English,C2,French,...,Turkish,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,junior,master,44000,"frontend-developer,full-stack-developer,c-c-de...",German,C2,English,C2,Spanish,...,French,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,senior,none,40000,"frontend-developer,backend-developer,php-devel...",English,C2,Romanian,C2,German,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,junior,apprenticeship,46000,"frontend-developer,backend-developer,full-stac...",German,C2,English,C1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,midlevel,master,75000,"devops-engineer,frontend-developer,backend-dev...",German,C2,English,C1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,False,midlevel,apprenticeship,101250,"cloud-engineer,frontend-developer,system-engin...",German,A2,English,B2,French,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,False,none,bachelor,62639,"product-manager,sales-manager,key-account-mana...",German,C1,English,C1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,False,junior,bachelor,99220,"c-c-developer,machine-learning-engineer,ui-ux-...",German,C2,English,C1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,False,junior,bachelor,73440,content-marketing-manager,German,C2,English,C1,Dutch,...,French,A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
jobs_df

,label,job_roles,seniorities,max_salary,min_degree,required_language_1_title,required_language_1_rating,required_language_1_must_have,required_language_2_title,required_language_2_rating,required_language_2_must_have
0,True,frontend-developer,"junior,midlevel",70000,none,German,C1,True,English,B2,True
1,True,frontend-developer,"junior,midlevel",70000,none,German,C1,True,English,B2,True
2,True,php-developer,"midlevel,senior",65000,none,German,C1,False,English,C1,True
3,True,frontend-developer,"junior,midlevel",70000,none,German,C1,True,English,B2,True
4,True,"backend-developer,full-stack-developer",midlevel,80000,bachelor,German,C1,True,English,C1,False
...,...,...,...,...,...,...,...,...,...,...,...
1995,False,sales-manager,"junior,midlevel,senior",75000,master,German,C1,True,NaN,NaN,NaN
1996,False,"sales-manager,presales-manager","none,junior",54000,none,German,C2,True,NaN,NaN,NaN
1997,False,"full-stack-developer,java-developer","senior,none,midlevel",82000,bachelor,German,C1,True,NaN,NaN,NaN
1998,False,frontend-developer,junior,54000,none,German,C1,True,English,B2,False


In [7]:
# src/feature_engineering.py

from typing import Dict
import numpy as np

def compare_language_levels(talent_level: str, required_level: str) -> int:
    levels = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']
    return levels.index(talent_level) - levels.index(required_level)

def engineer_features(talent: Dict, job: Dict) -> np.ndarray:
    features = []
    
    # Language match
    talent_languages = {lang['title']: lang['rating'] for lang in talent['languages']}
    job_languages = {lang['title']: lang['rating'] for lang in job['languages']}
    
    language_match_score = 0
    for lang, required_level in job_languages.items():
        if lang in talent_languages:
            talent_level = talent_languages[lang]
            if compare_language_levels(talent_level, required_level) >= 0:
                language_match_score += 1
    
    features.append(language_match_score / len(job_languages))
    
    # Job role match
    job_role_match = len(set(talent['job_roles']) & set(job['job_roles'])) / len(job['job_roles'])
    features.append(job_role_match)
    
    # Seniority match
    seniority_match = 1 if talent['seniority'] in job['seniorities'] else 0
    features.append(seniority_match)
    
    # Salary match
    salary_match = 1 if talent['salary_expectation'] <= job['max_salary'] else 0
    features.append(salary_match)
    
    # Degree match
    degree_levels = {'none': 0, 'bachelor': 1, 'master': 2, 'phd': 3}
    degree_match = 1 if degree_levels[talent['degree']] >= degree_levels[job['min_degree']] else 0
    features.append(degree_match)
    
    return np.array(features)


import sys
import numpy as np
sys.path.append('..')

# from src.data_processing import load_data, clean_data
# from src.feature_engineering import engineer_features
# from src.model import TalentJobMatcher

import numpy as np

# Prepare features and labels
data = load_data('data.json')
cleaned_data = clean_data(data)

X = np.array([engineer_features(item['talent'], item['job']) for item in cleaned_data])
y = np.array([item['label'] for item in cleaned_data])

KeyError: 'apprenticeship'

In [ ]:
{'talent': {'languages': [{'rating': 'C2', 'title': 'German'},
    {'rating': 'C2', 'title': 'English'},
    {'rating': 'B2', 'title': 'French'},
    {'rating': 'A2', 'title': 'Turkish'}],
   'job_roles': ['frontend-developer',
    'backend-developer',
    'full-stack-developer',
    'java-developer',
    'mobile-developer'],
   'seniority': 'junior',
   'salary_expectation': 48000,
   'degree': 'bachelor'}   {'title': 'English', 'rating': 'B2', 'must_have': True}],
   'job_roles': ['frontend-developer'],
   'seniorities': ['junior', 'midlevel'],
   'max_salary': 70000,
   'min_degree': 'none'},
  'label': True}

In [3]:
json_data = load_data('data.json')
json_data

[{'talent': {'languages': [{'rating': 'C2', 'title': 'German'},
    {'rating': 'C2', 'title': 'English'},
    {'rating': 'B2', 'title': 'French'},
    {'rating': 'A2', 'title': 'Turkish'}],
   'job_roles': ['frontend-developer',
    'backend-developer',
    'full-stack-developer',
    'java-developer',
    'mobile-developer'],
   'seniority': 'junior',
   'salary_expectation': 48000,
   'degree': 'bachelor'},
  'job': {'languages': [{'title': 'German', 'rating': 'C1', 'must_have': True},
    {'title': 'English', 'rating': 'B2', 'must_have': True}],
   'job_roles': ['frontend-developer'],
   'seniorities': ['junior', 'midlevel'],
   'max_salary': 70000,
   'min_degree': 'none'},
  'label': True},
 {'talent': {'languages': [{'rating': 'C2', 'title': 'German'},
    {'rating': 'C2', 'title': 'English'},
    {'rating': 'A2', 'title': 'Spanish'},
    {'rating': 'A2', 'title': 'French'}],
   'job_roles': ['frontend-developer',
    'full-stack-developer',
    'c-c-developer',
    'mobile-devel

In [8]:
import boto3

# Create an S3 client
s3 = boto3.client('s3')

# List all buckets
response = s3.list_buckets()
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  infra-service-cd-serverlessdeploymentbucket-wcmklmxg2mm8
  regn-ls-s3
  regn-ml-team
  regn-mvp-demo-bucket-1
  vodafn-ls-s3


In [9]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('test.db')

# Create a cursor object
cursor = conn.cursor()

# Create a table
cursor.execute('''CREATE TABLE IF NOT EXISTS users
                  (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)''')

# Insert a row of data
cursor.execute("INSERT INTO users (name, age) VALUES ('Alice', 30)")

# Save (commit) the changes
conn.commit()

# Query the database
cursor.execute("SELECT * FROM users")
print(cursor.fetchall())

# Close the connection
conn.close()

[(1, 'Alice', 30), (2, 'Alice', 30)]


In [10]:
import boto3
import json
import sqlite3

# Define the S3 bucket and prefix
BUCKET_NAME = 'regnwcc-ml-team'
PREFIX = FEEDBACK_PREFIX = 'ttt-interactions'

def migrate_s3_to_sqlite():
    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()

    # Create the tables if they don't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS chat_history (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT NOT NULL,
                        question TEXT NOT NULL,
                        response TEXT NOT NULL,
                        style TEXT,
                        number_of_references INTEGER,
                        score_threshold REAL,
                        sources TEXT,
                        timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    )''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS search_analytics (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT NOT NULL,
                        query TEXT NOT NULL,
                        result_count INTEGER NOT NULL,
                        has_click BOOLEAN DEFAULT FALSE,
                        timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    )''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS top_search_queries (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        query TEXT NOT NULL,
                        count INTEGER NOT NULL,
                        last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    )''')

    # Create indexes for improved query performance
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_search_analytics_username ON search_analytics(username)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_search_analytics_query ON search_analytics(query)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_top_search_queries_count ON top_search_queries(count DESC)')

    # Initialize S3 client
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')

    # Iterate over the objects in the S3 bucket
    for page in paginator.paginate(Bucket=BUCKET_NAME, Prefix=PREFIX):
        for obj in page.get('Contents', []):
            response = s3.get_object(Bucket=BUCKET_NAME, Key=obj['Key'])
            chat = json.loads(response['Body'].read().decode('utf-8'))

            # Insert data into the chat_history table
            cursor.execute('''INSERT INTO chat_history (username, question, response, style, number_of_references, score_threshold, sources, timestamp)
                              VALUES (?, ?, ?, ?, ?, ?, ?, ?)''', 
                           (chat['username'], chat['question'], chat['response'], chat.get('style'), chat.get('number_of_references'), chat.get('score_threshold'), json.dumps(chat.get('sources')), chat.get('timestamp')))

    # Save (commit) the changes
    conn.commit()

    # Close the connection
    conn.close()

# Run the migration
migrate_s3_to_sqlite()


# test the migration
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

# Query the chat_history table
cursor.execute("SELECT * FROM chat_history")
print(cursor.fetchall())

# convert to df
import pandas as pd


[(1, 'admin', 'Why is there a significantly lower proportion of dedicated CCM resources on the buy-side compared to the sell-side, and what impact does this have on contract management outcomes?', '*Disclaimer: The following response is based on WorldCC data but is not legal advice.*\n\nThere are several factors that contribute to the lower proportion of dedicated Contract and Commercial Management (CCM) resources on the buy-side compared to the sell-side, and these differences have direct implications on contract management outcomes.\n\n### Factors for Lower Proportion of Dedicated CCM Resources on the Buy-side\n\n1. **Developmental Stage and Accountability**:\n   Buy-side organizations are generally less advanced in their development of CCM competency. This is reflected in the significantly lower likelihood of having clear ownership or accountability for CCM. For instance, 27% of buy-side organizations have no dedicated CCM resources compared to just 9% on the sell-side [1].\n\n2. **

In [6]:
# convert to df
import pandas as pd

df = pd.read_sql_query("SELECT * FROM chat_history", conn)
df

c:\Users\Freedom\Anaconda3\envs\srb_rec\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,id,username,question,response,style,number_of_references,score_threshold,sources,timestamp
0,1,admin,Why is there a significantly lower proportion ...,*Disclaimer: The following response is based o...,Style 1 - Tim Style Based on Formal Structure:...,7,0.6,"{""7"": 0.99999744, ""8"": 0.9998118, ""12"": 0.9996...",None
1,2,afurner@worldcc.com,I was wondering if you had any metrics based o...,*Disclaimer: The following response is based o...,Style 1 - Tim Style Based on Formal Structure:...,7,0.6,"{""328"": 0.9081302, ""278"": 0.86013436, ""276"": 0...",None
2,3,afurner@worldcc.com,What is the cost of a contract in 2024?,*Disclaimer: I cannot answer the question base...,Style 1 - Tim Style Based on Formal Structure:...,7,0.6,{},None
3,4,afurner@worldcc.com,What is the latest thinking on the ethics and ...,*Disclaimer: The response is based on the Worl...,Style 1 - Tim Style Based on Formal Structure:...,7,0.6,"{""11"": 0.88856554, ""26"": 0.82403755}",None
4,5,afurner@worldcc.com,What is the latest thinking on the ethics and ...,*Disclaimer: The following response is based o...,Style 1 - Tim Style Based on Formal Structure:...,7,0.6,"{""91"": 0.81626, ""26"": 0.7525585, ""99"": 0.71213...",None
...,...,...,...,...,...,...,...,...,...
75,76,jdavis@worldcc.com,Do the key barriers to adopting new technology...,*Disclaimer: I can't answer the question based...,Style 1 - Tim Style Based on Formal Structure:...,7,0.8,{},None
76,77,jdavis@worldcc.com,Can you give me some advice on data monetisati...,*Disclaimer: I can't answer the question based...,Style 1 - Tim Style Based on Formal Structure:...,7,0.6,{},None
77,78,jdavis@worldcc.com,Can you advise me on sun protection for July i...,*Disclaimer: I am unable to answer this questi...,Style 1 - Tim Style Based on Formal Structure:...,7,0.6,{},None
78,79,jdavis@worldcc.com,Can you advise me on sun protection for July i...,*Disclaimer: I can only provide answers about ...,Style 3 - General Best Practice Legal Research...,7,0.6,{},None


In [12]:
df.iloc[1]['question']

'I was wondering if you had any metrics based on volume of contract intake requests to the ratio of contract professionals (contract managers/contract specialist) do the work.  For example, my team of Contract Professionals handle over 60% of the contract intake request for the legal team.  Is there a way for me to benchmark on what other companies are doing in that space?  We separate attorney work, contract professional work, and operations work.'